# Goal: 
#### To process semi-structured JSON credit_report_sample file and create a reusable function that extracts meaningful risk features.

In [393]:
import pandas as pd
import json
from datetime import datetime, timedelta
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

## Initial Data Loading & Exploration

In [38]:
base_credit_report_data = pd.read_json('../data/credit_report_sample.json')
base_credit_report_data

,application_id,data
0,9711360,{'consumerfullcredit': {'subjectlist': {'refer...
1,9714953,{'consumerfullcredit': {'subjectlist': {'refer...
2,9714978,{'consumerfullcredit': {'subjectlist': {'refer...


The data is provided as a list of dictionaries, each containing an _application_id_ and a _data_ key holding the credit report.

__Assumption:__ I will assume the application_id is the unique identifier for each customer record. My final output will retain this column along with the extracted features so that it can be easily joined back to other customer-level data.

In [39]:
base_credit_report_data.data.iloc[0]

{'consumerfullcredit': {'subjectlist': {'reference': '128566',
   'consumerid': '128566',
   'searchoutput': 'XXX '},
  'accountrating': {'noofotheraccountsbad': '0',
   'noofotheraccountsgood': '3',
   'noofretailaccountsbad': '0',
   'noofretailaccountsgood': '2',
   'nooftelecomaccountsbad': '0',
   'noofautoloanaccountsbad': '0',
   'noofautoloanccountsgood': '0',
   'noofhomeloanaccountsbad': '0',
   'nooftelecomaccountsgood': '0',
   'noofhomeloanaccountsgood': '0',
   'noofjointloanaccountsbad': '0',
   'noofstudyloanaccountsbad': '0',
   'noofcreditcardaccountsbad': '0',
   'noofjointloanaccountsgood': '0',
   'noofstudyloanaccountsgood': '0',
   'noofcreditcardaccountsgood': '1',
   'noofpersonalloanaccountsbad': '0',
   'noofpersonalloanaccountsgood': '1'},
  'enquirydetails': {'productid': '45',
   'matchingrate': '90',
   'subscriberenquiryengineid': '5012874225',
   'subscriberenquiryresultid': '6381470'},
  'guarantorcount': {'accounts': '0', 'guarantorssecured': '0'},
  

### Formatting the df to a better view

In [479]:
formatted_base_credit_report_data = pd.DataFrame()
for i in range(len(base_credit_report_data)):
    # Creating an intermediate dataframe with all the keys in the data dictionary
    temp_df_1 = pd.DataFrame(base_credit_report_data.data.iloc[i])
    temp_df_1['application_id'] = base_credit_report_data.application_id.iloc[i]

    # Pivoting the intermediate dataframe to create a single row for every application_id
    temp_df_2 = temp_df_1.reset_index().pivot(columns=['index'],
                                                index=['application_id'],
                                                values=['consumerfullcredit']).reset_index()
    temp_df_2.columns = ['application_id'] + temp_df_2['consumerfullcredit'].keys().tolist()

    formatted_base_credit_report_data = pd.concat([formatted_base_credit_report_data, temp_df_2])

formatted_base_credit_report_data

,application_id,accountmonthlypaymenthistory,accountmonthlypaymenthistoryheader,accountrating,creditaccountsummary,creditagreementsummary,deliquencyinformation,employmenthistory,enquirydetails,enquiryhistorytop,guarantorcount,guarantordetails,personaldetailssummary,subjectlist,telephonehistory,identificationhistory
0,9711360,"[{'m01': '#', 'm02': '3', 'm03': '0', 'm04': '...","{'mh01': '2020 AUG', 'mh02': '2020 JUL', 'mh03...","{'noofotheraccountsbad': '0', 'noofotheraccoun...","{'rating': '13', 'amountarrear': '24,041.00', ...","[{'currency': 'NGN', 'accountno': '03245626361...","{'accountno': '2150224012', 'periodnum': '2014...","[{'occupation': 'PUBLIC SERVANTS', 'updatedate...","{'productid': '45', 'matchingrate': '90', 'sub...","[{'daterequested': '06/08/2020 18:05:36', 'enq...","{'accounts': '0', 'guarantorssecured': '0'}","{'guarantorgender': None, 'guarantorotherid': ...","{'gender': 'Male', 'header': 'PERSONAL DETAILS...","{'reference': '128566', 'consumerid': '128566'...","[{'homenoupdatedondate': '08/04/2016', 'homete...",NaN
0,9714953,"[{'m01': '#', 'm02': '#', 'm03': '#', 'm04': '...","{'mh01': '2020 SEP', 'mh02': '2020 AUG', 'mh03...","{'noofotheraccountsbad': '0', 'noofotheraccoun...","{'rating': '2', 'amountarrear': '0.00', 'amoun...","[{'currency': 'NGN', 'accountno': '0494922010'...","{'accountno': 'XXX', 'periodnum': '20170430', ...","[{'occupation': 'CIVIL SERVANT', 'updatedate':...","{'productid': '45', 'matchingrate': '90', 'sub...","[{'daterequested': '31/07/2020 09:18:19', 'enq...","{'accounts': '0', 'guarantorssecured': '0'}","{'guarantorgender': None, 'guarantorotherid': ...","{'gender': 'Female', 'header': 'PERSONAL DETAI...","{'reference': '58793', 'consumerid': '58793', ...","[{'homenoupdatedondate': '08/04/2016', 'homete...","[{'updatedate': '21/11/2014', 'updateondate': ..."
0,9714978,"[{'m01': '#', 'm02': '#', 'm03': '#', 'm04': '...","{'mh01': '2020 SEP', 'mh02': '2020 AUG', 'mh03...","{'noofotheraccountsbad': '0', 'noofotheraccoun...","{'rating': '109', 'amountarrear': '12,000.00',...","[{'currency': 'NGN', 'accountno': '02380228104...","{'accountno': '4181042465', 'periodnum': '2020...","[{'occupation': 'STUDENT'}, {'occupation': 'ST...","{'productid': '45', 'matchingrate': '90', 'sub...","[{'daterequested': '31/07/2020 08:47:15', 'enq...","{'accounts': '0', 'guarantorssecured': '0'}","{'guarantorgender': None, 'guarantorotherid': ...","{'gender': 'Female', 'header': 'PERSONAL DETAI...","{'reference': '17688366', 'consumerid': '17688...","[{'mobiletelephonenumber': 'XXX'}, {'mobiletel...",NaN


TODO: Edit below based on features made

## Feature Engineering Strategy
After an initial review of the credit_report_sample.json data, I've identified sections that are rich with information that will be useful in credit risk modelling. Below is the strategy I've come up with to extract features:

1. ___Personal and Demographic Information___

    __Source Fields:__ personaldetailssummary, employmenthistory

    __Rationale:__ To add context on applicant stability and life stage, I will extract key demographic features. While often less predictive than behavioral data, these features add valuable dimensionality to the model.

    __From this data, I will extract:__
        age (calculated from date of birth).
        A binary flag for whether the customer is a foreigner or not (is_foreigner).
        count_of_occupations_listed (number of different types of occupations listed in the employment history)
        occupation_type (latest occupation type of the customer - based on the entries which have an updatedate mentioned)
   
3. ___Delinquency and Negative Credit Events___

   __Source Fields:__ accountrating, creditaccountsummary, deliquencyinformation

   __Rationale:__ Next, I will identify clear indicators of financial distress. These sections provide high-level summaries and specific flags for negative events, which are often highly predictive.

   __Features to be created will include:__
        A simple binary flag, is_currently_delinquent.
        Apart from the derived feature here, the elements within each field seem valuable by themself.

4. ___Credit Seeking Behavior___

    __Source Field:__ enquiryhistorytop

    __Rationale:__ To understand recent credit-seeking activity, I will analyze the enquiry history. A recent increase in enquiries is a well-established risk signal, often indicating immediate financial need.

    __This will involve creating features like:__
        count_of_enquiries_last_3_months and last_6_months.
        days_since_last_enquiry to capture recency.


6. ___Tradeline and Repayment Behavior___

   __Source Fields:__ accountmonthlypaymenthistory, creditagreementsummary

   __Rationale:__ The core of my analysis will focus on these sections, as they detail the customer's past performance and current debt load. Understanding this behavior is fundamental to predicting future risk.

   __Key features I will build include:__
        Aggregate metrics like total_active_accounts and age_of_oldest_tradeline.
        Loan type analysis, such as the ratio_of_secured_vs_unsecured_loans.
        Current debt burden metrics, like the aggregate_monthly_payment_burden.


___Note: Since all the date fields end sometime in Aug 2020, i will use the comparison date as 1st Sept 2020___

### 1. Personal and Demographic Information

#### Goal:

The first goal is to understand the elements of _personaldetailssummary_ & _employmenthistory_ fields, the different elements in each of them and the features that can be derived

#### Exploring a sample record

In [71]:
formatted_base_credit_report_data.personaldetailssummary.iloc[0]

{'gender': 'Male',
 'header': 'PERSONAL DETAILS SUMMARY: XXX',
 'surname': 'XXX',
 'birthdate': '06/05/1991',
 'firstname': 'XXX',
 'otheridno': None,
 'cellularno': 'XXX',
 'consumerid': '128566',
 'dependants': '0',
 'othernames': 'XXX',
 'pencomidno': None,
 'nationality': 'Nigeria',
 'emailaddress': None,
 'nationalidno': None,
 'employerdetail': 'ALL MILITARY STAFFS',
 'postaladdress1': 'XXX ',
 'postaladdress2': 'KADUNA STATE NIGERIA',
 'postaladdress3': None,
 'postaladdress4': {'xml:space': 'preserve', '__content__': ' '},
 'hometelephoneno': 'XXX',
 'propertyownedtype': None,
 'bankverificationno': 'XXX',
 'residentialaddress1': 'XXX ',
 'residentialaddress2': 'KADUNA STATE  NIGERIA',
 'residentialaddress3': None,
 'residentialaddress4': {'xml:space': 'preserve', '__content__': ' '}}

Fields from _personaldetailssummary_ that can be taken directly are: gender, birthdate, dependents, nationality, propertyownedtype

Derived features:
    _age_: as of today from birthdate
    _is_foreigner_: boolean having 1 if nationality is not 'Nigeria'

In [81]:
formatted_base_credit_report_data.employmenthistory.iloc[0]

[{'occupation': 'PUBLIC SERVANTS',
  'updatedate': '16/09/2016',
  'updateondate': '16/09/2016',
  'employerdetail': 'ALL MILITARY STAFFS'},
 {'occupation': 'PUBLIC SERVANTS'},
 {'occupation': 'PUBLIC SERVANTS'},
 {'occupation': 'Army Police Defence', 'employerdetail': 'Nigerian navy'},
 {'occupation': 'PUBLIC SERVANTS'}]

Most valuable field in _employmenthistory_ is occupation.

Derived features: 
    _count_of_occupations_listed_: total number of unique types of values the 'occupation' field has for a given customer
    _occupation_type_: the occupation value in the first entry for a given customer. Rationale for this is that I am assuming it indicates the latest entry for that customer

#### Helper Function to generate above features

In [466]:
def extract_personal_demographic_info(df_subset):
    """This function extracts the personal and demographic information from the dataframe given"""
    # df_subset will only have application_id, personaldetailssummary, employmenthistory as the columns

    ### Values from personaldetailssummary
    # Create intermediate dataframe for extracting elements from personaldetailssummary field
    df_personal = df_subset['personaldetailssummary'].apply(pd.Series)
    df_personal['application_id'] = df_subset.application_id.tolist()

    required_personal_features = ['gender', 'birthdate', 'dependants', 'nationality', 'propertyownedtype']
    
    # Keeping only required fields from extracted columns
    df_personal_reqd = df_personal[required_personal_features + ['application_id']]

    # Formatting the column values and deriving features
    # Age
    df_personal_reqd['birthdate'] = pd.to_datetime(df_personal_reqd['birthdate'], format="%d/%m/%Y")
    df_personal_reqd['age'] = round((datetime.now() - df_personal_reqd.birthdate).dt.days/365, 0)

    # Nationality/Foreigner
    df_personal_reqd['is_foreigner'] = np.where(df_personal_reqd['nationality']!='Nigeria',
                                                1, 0)

    # Dropping birthdate and nationality columns as age and is_foreigner keys have been derived
    df_personal_reqd.drop(['birthdate', 'nationality'], axis=1, inplace=True)
    
    ## Values from employmenthistory
    # Creating intermediate dataframes for extracting elements from employmenthistory field
    df_employment_1 = df_subset['employmenthistory'].apply(pd.Series)
    df_employment_1['application_id'] = df_subset.application_id.tolist()

    # Another intermediate dataframe is required in this case as each row of the subset dataframe has more than 1 
    # dictionaries
    df_employment_2 = pd.DataFrame()
    for i in range(0, len(df_employment_1.columns)-1):
        temp_df_employment = df_employment_1[i].apply(pd.Series)

        # The below if condition is used to store the latest occupation type of a customer.
        # This is based on the assumption that the dictionary having update date signals the latest occupation of the 
        # customer
        if i==0:
            temp_df_employment['occupation_type'] = temp_df_employment.occupation
        temp_df_employment['application_id'] = df_employment_1.application_id.tolist()
        df_employment_2 = pd.concat([df_employment_2, temp_df_employment])

    df_employment = df_employment_2.groupby(by=['application_id'], 
                                            as_index=False).aggregate(count_of_occupations_listed=('occupation', 'nunique'))
    df_employment = df_employment.merge(df_employment_2[['application_id', 'occupation_type']
                                                        ][~df_employment_2.occupation_type.isnull()], on="application_id",
                                        how="left")
    
    return df_personal_reqd.merge(df_employment, on="application_id", how="left")

#### Testing Helper Function

In [467]:
personal_demo_info = extract_personal_demographic_info(formatted_base_credit_report_data[['application_id', 
                                                                                          'personaldetailssummary', 
                                                                                          'employmenthistory']])
personal_demo_info

,gender,dependants,propertyownedtype,application_id,age,is_foreigner,count_of_occupations_listed,occupation_type
0,Male,0,None,9711360,34.0,0,2,PUBLIC SERVANTS
1,Female,0,None,9714953,40.0,0,2,CIVIL SERVANT
2,Female,0,None,9714978,42.0,0,1,STUDENT


### 2. Delinquency and Negative Credit Events

#### Goal:

Here I've delved into accountrating, creditaccountsummary, deliquencyinformation fields and fetched the important features that could be used further

#### Exploring a sample record

In [468]:
formatted_base_credit_report_data.accountrating.iloc[0]

{'noofotheraccountsbad': '0',
 'noofotheraccountsgood': '3',
 'noofretailaccountsbad': '0',
 'noofretailaccountsgood': '2',
 'nooftelecomaccountsbad': '0',
 'noofautoloanaccountsbad': '0',
 'noofautoloanccountsgood': '0',
 'noofhomeloanaccountsbad': '0',
 'nooftelecomaccountsgood': '0',
 'noofhomeloanaccountsgood': '0',
 'noofjointloanaccountsbad': '0',
 'noofstudyloanaccountsbad': '0',
 'noofcreditcardaccountsbad': '0',
 'noofjointloanaccountsgood': '0',
 'noofstudyloanaccountsgood': '0',
 'noofcreditcardaccountsgood': '1',
 'noofpersonalloanaccountsbad': '0',
 'noofpersonalloanaccountsgood': '1'}

All the fields in _accountrating_ can be used in the model. The number of good and bad accounts in any sector play an important role in predicting the customer's risk taking ability.

In [469]:
formatted_base_credit_report_data.creditaccountsummary.iloc[0]

{'rating': '13',
 'amountarrear': '24,041.00',
 'amountarrear1': '0.00',
 'totalaccounts': '7',
 'totalaccounts1': '0',
 'lastjudgementdate': '-',
 'lastjudgementdate1': '-',
 'totalaccountarrear': '2',
 'totalaccountarrear1': '0',
 'totaljudgementamount': '0',
 'totaloutstandingdebt': '105,435.00',
 'totaljudgementamount1': '0',
 'totaloutstandingdebt1': '0.00',
 'totaldishonouredamount': '0.00',
 'totalmonthlyinstalment': '77,404.00',
 'totalnumberofjudgement': '0',
 'totaldishonouredamount1': '0.00',
 'totalmonthlyinstalment1': '0.00',
 'totalnumberofjudgement1': '0',
 'totalnumberofdishonoured': '0',
 'totalnumberofdishonoured1': '0',
 'totalaccountingodcondition': '0',
 'totalaccountingodcondition1': '0'}

For _creditaccountsummary_ field, elements like rating, amountarrear, totalaccounts, totalaccountarrear, totaloutstandingdebt, totaldishonouredamount, totalmonthlyinstalment are valuable

In [470]:
formatted_base_credit_report_data.deliquencyinformation.iloc[0]

{'accountno': '2150224012',
 'periodnum': '20140930',
 'subscribername': 'First City Monument Bank Ltd Lagos',
 'monthsinarrears': '13'}

In _deliquencyinformation_, i will retain the delinquent subscriber name and monthsinarrears 

#### Helper Function to generate above features

In [471]:
def extract_credit_summary_delinquency_data(df_subset):
    """This function extracts the credit summary data and delinquency related data"""

    ### Values from accountrating
    # Create intermediate dataframe for extracting elements from accountrating field
    df_account_rating = df_subset['accountrating'].apply(pd.Series)
    df_account_rating['application_id'] = df_subset.application_id.tolist()
    # All fields of accountrating will be retained

    ## Values from creditaccountsummary
    # Create intermediate dataframe for extracting elements from accountrating field
    df_credit_summary = df_subset['creditaccountsummary'].apply(pd.Series)
    df_credit_summary['application_id'] = df_subset.application_id.tolist()
    
    required_credit_summary_features = ['rating', 'amountarrear', 'totalaccounts', 'totalaccountarrear', 
                                         'totaloutstandingdebt', 'totaldishonouredamount', 'totalmonthlyinstalment']
    
    # Keeping only required fields from extracted columns
    df_credit_summary_reqd = df_credit_summary[required_credit_summary_features + ['application_id']]

    # is_currently_delinquent - if totalaccountarrear>0, then 1 else 0
    df_credit_summary_reqd['is_currently_delinquent'] = np.where(df_credit_summary_reqd['totalaccountarrear'].astype(int)>0,
                                                                 1, 0)

    ## Values from deliquencyinformation
    # Create intermediate dataframe for extracting elements from deliquencyinformation field
    df_delinquency = df_subset['deliquencyinformation'].apply(pd.Series)
    df_delinquency['application_id'] = df_subset.application_id.tolist()
    
    required_delinquency_features = ['subscribername', 'monthsinarrears']
    
    # Keeping only required fields from extracted columns
    df_delinquency_reqd = df_delinquency[required_delinquency_features + ['application_id']]
    df_delinquency_reqd.rename({'subscribername': 'delinquentsubscribername',
                                'monthsinarrears': 'delinquentmonthsinarrears'}, inplace=True)

    df_credit_delinquent = df_account_rating.merge(df_credit_summary_reqd, on="application_id", how="left")
    df_credit_delinquent = df_credit_delinquent.merge(df_delinquency_reqd, on="application_id", how="left")

    return df_credit_delinquent

#### Testing Helper Function

In [472]:
credit_summary_delinquent_data = extract_credit_summary_delinquency_data(formatted_base_credit_report_data)
credit_summary_delinquent_data

,noofotheraccountsbad,noofotheraccountsgood,noofretailaccountsbad,noofretailaccountsgood,nooftelecomaccountsbad,noofautoloanaccountsbad,noofautoloanccountsgood,noofhomeloanaccountsbad,nooftelecomaccountsgood,noofhomeloanaccountsgood,...,rating,amountarrear,totalaccounts,totalaccountarrear,totaloutstandingdebt,totaldishonouredamount,totalmonthlyinstalment,is_currently_delinquent,subscribername,monthsinarrears
0,0,3,0,2,0,0,0,0,0,0,...,13,"24,041.00",7,2,"105,435.00",0.00,"77,404.00",1,First City Monument Bank Ltd Lagos,13
1,0,3,0,12,0,0,0,0,0,0,...,2,0.00,17,1,"294,770.00",0.00,"132,176.00",1,First City Monument Bank Ltd Lagos,2
2,0,1,1,1,0,0,0,0,0,0,...,109,"12,000.00",3,1,"110,919.00",0.00,"7,000.00",1,Branch International Financial Services Limited,109


### 3. Credit Seeking Behavior

#### Goal:

The goal here is to derive features from _enquiryhistorytop_ field that will reflect the credit-seeking activity of the customer

#### Exploring a sample record

In [473]:
formatted_base_credit_report_data.enquiryhistorytop.iloc[0]

[{'daterequested': '06/08/2020 18:05:36',
  'enquiryreason': 'Credit scoring of the client by credit bureau',
  'subscribername': 'Fairmoney - FairMoneyAPI',
  'subscriberenquiryresultid': '6050316'},
 {'daterequested': '01/08/2020 06:59:09',
  'enquiryreason': 'Credit scoring of the client by credit bureau',
  'subscribername': 'Fairmoney - FairMoneyAPI',
  'subscriberenquiryresultid': '5983396'},
 {'daterequested': '08/07/2020 15:58:44',
  'enquiryreason': 'application of existing credit by a borrower',
  'subscribername': 'NewEdge Finance Limited  - transsnet',
  'subscriberenquiryresultid': '5680296'},
 {'daterequested': '03/07/2020 22:58:39',
  'enquiryreason': 'Credit scoring of the client by credit bureau',
  'subscribername': 'Fairmoney - FairMoneyAPI',
  'subscriberenquiryresultid': '5637826'},
 {'daterequested': '04/06/2020 07:55:58',
  'enquiryreason': 'Credit scoring of the client by credit bureau',
  'subscribername': 'Fairmoney - FairMoneyAPI',
  'subscriberenquiryresulti

From the _enquiryhistorytop_ field, total_enquiries, count_of_enquiries_last_3_months and count_of_enquiries_last_6_months. days_since_last_enquiry can be calculated. Enquiry recency correlates well with the loan repayment and loan taking ability of the customer making these features 

#### Helper Function to generate above features

In [474]:
def extract_enquiry_data(df_subset):
    """This function extracts all features from enquiry data"""

    ## Values from enquiryhistorytop
    # Creating intermediate dataframes for extracting elements from enquiryhistorytop field
    df_enquiry_1 = df_subset['enquiryhistorytop'].apply(pd.Series)
    df_enquiry_1['application_id'] = df_subset.application_id.tolist()

    # Another intermediate dataframe is required in this case as each row of the subset dataframe has more than 1 
    # dictionaries
    df_enquiry = pd.DataFrame()
    for i in range(0, len(df_enquiry_1.columns)-1):
        temp_df_enquiry = df_enquiry_1[i].apply(pd.Series)
        temp_df_enquiry['application_id'] = df_enquiry_1.application_id.tolist()
        df_enquiry = pd.concat([df_enquiry, temp_df_enquiry])

    # Comparison date: assumed to be 1st Sept 2020
    comparison_date = datetime(2020, 9, 1).date()

    # To calculate enquiries in last 3 months, the min date would be 91 days before comparison date
    last_3_months_min_date = comparison_date - timedelta(days=91)

    # To calculate enquiries in last 6 months, the min date would be 183 days before comparison date
    last_6_months_min_date = comparison_date - timedelta(days=183)

    # Formating values of the df_enquiry_2 dataframe
    df_enquiry['daterequested'] = pd.to_datetime(df_enquiry.daterequested, format="%d/%m/%Y %H:%M:%S")

    # Rows without daterequested cannot be used for any of the features so they have to be droped
    df_enquiry = df_enquiry[~df_enquiry['daterequested'].isnull()]

    df_enquiry['daterequested'] = pd.to_datetime(df_enquiry.daterequested, errors='coerce').dt.date
    
    # Creating additional boolean columns for enquiries in last 3 months and in last 6 months
    df_enquiry['is_last_3_months_enquiry'] = np.where(df_enquiry['daterequested']>=last_3_months_min_date,
                                                      1, 0)
    df_enquiry['is_last_6_months_enquiry'] = np.where(df_enquiry['daterequested']>=last_6_months_min_date,
                                                      1, 0)
    
    # Calculating the days since enquiry
    df_enquiry['days_since_enquiry'] = (pd.Timestamp(comparison_date).normalize() - 
                                        pd.to_datetime(df_enquiry['daterequested'])).dt.days 
    
    df_enquiry_reqd = df_enquiry.groupby(by=['application_id'], 
                                         as_index=False
                                         ).aggregate(total_enquiries=('subscriberenquiryresultid', 'count'),
                                                     unique_subscriber_count=('subscribername', 'nunique'),
                                                     count_of_enquiries_last_3_months=('is_last_3_months_enquiry', 'sum'),
                                                     count_of_enquiries_last_6_months=('is_last_6_months_enquiry', 'sum'),
                                                     days_since_last_enquiry=('days_since_enquiry', 'min'))
    
    return df_enquiry_reqd

#### Testing Helper Function

In [475]:
enquiry_data = extract_enquiry_data(formatted_base_credit_report_data)
enquiry_data

,application_id,total_enquiries,unique_subscriber_count,count_of_enquiries_last_3_months,count_of_enquiries_last_6_months,days_since_last_enquiry
0,9711360,15,2,5,13,26
1,9714953,10,4,6,7,32
2,9714978,10,3,4,8,32


### 4. Tradeline and Repayment Behavior

#### Goal:

Here the most important features will be derived which will be based on _accountmonthlypaymenthistory_

#### Exploring a sample record

In [476]:
formatted_base_credit_report_data.accountmonthlypaymenthistory.iloc[0]

[{'m01': '#',
  'm02': '3',
  'm03': '0',
  'm04': '0',
  'm05': '0',
  'm06': '0',
  'm07': '36',
  'm08': '36',
  'm09': '0',
  'm10': '31',
  'm11': '0',
  'm12': '9',
  'm13': '1',
  'm14': '0',
  'm15': '0',
  'm16': '3',
  'm17': '5',
  'm18': '0',
  'm19': '4',
  'm20': '4',
  'm21': '1',
  'm22': '0',
  'm23': '4',
  'm24': '3',
  'header': 'Details of Credit Agreement with "First City Monument Bank Ltd Lagos" for Account Number: 0324562636176001',
  'currency': 'NGN',
  'accountno': '0324562636176001',
  'tablename': 'Consumer24MonthlyPayment',
  'closeddate': '01/11/2020',
  'accountnote': None,
  'displaytext': 'Consumer 24 Monthly Payment',
  'loanduration': '1170 Day(s)',
  'subaccountno': None,
  'accountstatus': 'Open',
  'amountoverdue': '22,441.39',
  'subscribername': 'First City Monument Bank Ltd Lagos',
  'lastpaymentdate': '01/08/2020',
  'lastupdateddate': '07/08/2020',
  'currentbalanceamt': '82,733.58',
  'dateaccountopened': '15/01/2016',
  'openingbalanceamt':

From the tradeline level data following features can be derived which portray alot of information on the customer: highest_dpd_overall, highest_dpd_last_3_months, highest_dpd_last_6_months, cycle_count_dpd_greater_than_7, cycle_count_dpd_greater_than_15, tradeline_count, types_of_loans, total_unique_lenders, avg_loan_duration_days, bureau_account_age. 

Further based on the loans still open as per the tradelines, these additional features can be derived: total_opening_balance, mean_opening_balance, total_current_balance, mean_current_balance, total_monthly_installment, total_amount_overdue, active_tradelines_count.

#### Helper Function to generate above features

In [477]:
def extract_tradeline_data(df_subset):
    """This function extracts features from tradeline data which shows performance of the customer in 
    each loan/credit line he/she has taken"""

    ## Values from accountmonthlypaymenthistory
    # Creating intermediate dataframes for extracting elements from accountmonthlypaymenthistory field
    df_tradeline_1 = df_subset['accountmonthlypaymenthistory'].apply(pd.Series)
    df_tradeline_1['application_id'] = df_subset.application_id.tolist()

    # Another intermediate dataframe is required in this case as each row of the subset dataframe has more than 1 
    # dictionaries
    df_tradeline = pd.DataFrame()
    for i in range(0, len(df_tradeline_1.columns)-1):
        temp_df_tradeline = df_tradeline_1[i].apply(pd.Series)
        temp_df_tradeline['application_id'] = df_tradeline_1.application_id.tolist()
        df_tradeline = pd.concat([df_tradeline, temp_df_tradeline])

    if 0 in df_tradeline.columns:
        df_tradeline.drop([0], axis=1, inplace=True)

    # Format date columns
    df_tradeline['closeddate'] = pd.to_datetime(df_tradeline['closeddate'], format="%d/%m/%Y").dt.date
    df_tradeline['lastpaymentdate'] = pd.to_datetime(df_tradeline['lastpaymentdate'], format="%d/%m/%Y").dt.date
    df_tradeline['lastupdateddate'] = pd.to_datetime(df_tradeline['lastupdateddate'], format="%d/%m/%Y").dt.date
    df_tradeline['dateaccountopened'] = pd.to_datetime(df_tradeline['dateaccountopened'], format="%d/%m/%Y").dt.date

    # Tradelines without dateaccountopened mean insufficient data, so dropping those rows
    df_tradeline = df_tradeline[~df_tradeline.dateaccountopened.isnull()]

    # For every tradeline, highest_dpd_overall, highest_dpd_last_3_months, highest_dpd_last_6_months, 
    # cycle_count_dpd_greater_than_7, cycle_count_dpd_greater_than_15 has to be calculated
    df_tradeline.reset_index(drop=True, inplace=True)
    for index, row in df_tradeline.iterrows():
        dpd_value_list = [value for col_name, value in row.items() if re.search(r'\d', col_name)]
        dpd_overall = [int(value) for value in dpd_value_list if (value!='#')]
        dpd_3m = [int(value) for value in dpd_value_list[:3] if (value!='#')]
        dpd_6m = [int(value) for value in dpd_value_list[:6] if (value!='#')]
    
        df_tradeline.loc[index, 'highest_dpd_overall'] = max(dpd_overall) if dpd_overall else 0
        df_tradeline.loc[index, 'highest_dpd_last_3_months'] = max(dpd_3m) if dpd_3m else 0
        df_tradeline.loc[index, 'highest_dpd_last_6_months'] = max(dpd_6m) if dpd_6m else 0

        df_tradeline.loc[index, 'cycle_count_dpd_greater_than_7'] = len([value for value in dpd_overall if value > 7]
                                                                        ) if dpd_overall else 0
        df_tradeline.loc[index, 'cycle_count_dpd_greater_than_15'] = len([value for value in dpd_overall if value > 15]
                                                                         ) if dpd_overall else 0

    # Formating important numeric and string fields
    df_tradeline['indicatordescription'] = df_tradeline['indicatordescription'].str.lower()
    df_tradeline['openingbalanceamt'] = df_tradeline['openingbalanceamt'].apply(lambda x: float(x.replace(",", ""))
                                                                                if type(x)==str else x)
    df_tradeline['currentbalanceamt'] = df_tradeline['currentbalanceamt'].apply(lambda x: float(x.replace(",", ""))
                                                                                if type(x)==str else x)
    df_tradeline['monthlyinstalmentamt'] = df_tradeline['monthlyinstalmentamt'].apply(lambda x: float(x.replace(",", "")) 
                                                                                      if type(x)==str else x)
    df_tradeline['amountoverdue'] = df_tradeline['amountoverdue'].apply(lambda x: float(x.replace(",", "")) 
                                                                                      if type(x)==str else x)
    df_tradeline['loanduration'] = df_tradeline['loanduration'].apply(lambda x: int(x.split(' ')[0]) if (x is not None) and 
                                                                      (re.search(r'\d', x.split(" ")[0])) else None)
    
    # Creating a column to identify open/active loans
    df_tradeline['is_active'] = np.where(df_tradeline['accountstatus'].str.lower().isin(['open', None]),
                                         1, 0)

    # First intermediate features dataframe to calculate days past due (DPD) and overall tradeline summary features
    df_tradeline_features_v1 = df_tradeline.groupby(by=['application_id'], 
                                                     as_index=False
                                                     ).aggregate(highest_dpd_overall=('highest_dpd_overall', 'max'),
                                                                 highest_dpd_last_3_months=('highest_dpd_last_3_months', 
                                                                                            'max'),
                                                                 highest_dpd_last_6_months=('highest_dpd_last_6_months',
                                                                                            'max'),
                                                                 cycle_count_dpd_greater_than_7=(
                                                                     'cycle_count_dpd_greater_than_7', 'sum'),
                                                                 cycle_count_dpd_greater_than_15=(
                                                                     'cycle_count_dpd_greater_than_15', 'sum'),
                                                                 tradeline_count=('accountno', 'count'),
                                                                 types_of_loans=('indicatordescription', 'nunique'),
                                                                 total_unique_lenders=('subscribername', 'nunique'),
                                                                 avg_loan_duration_days=('loanduration', 'mean'),
                                                                 min_open_date=('dateaccountopened', 'min'))
    # Comparison date: assumed to be 1st Sept 2020
    comparison_date = datetime(2020, 9, 1).date()

    # Calculating bureau_account_age in years based on minimum account open date and comparison date assumed above
    df_tradeline_features_v1['bureau_account_age_years'] = round(((pd.Timestamp(comparison_date).normalize() - 
                                                              pd.to_datetime(df_tradeline_features_v1.min_open_date)
                                                             ).dt.days)/365, 0)
    df_tradeline_features_v1.drop(['min_open_date'], axis=1, inplace=True)

    # Second intermediate features dataframe, features derived only on active/open loans are calculated below
    df_tradeline_features_v2 = df_tradeline[df_tradeline.is_active==1
                                             ].groupby(by=['application_id'], 
                                                       as_index=False
                                                       ).aggregate(total_opening_balance=('openingbalanceamt', 'sum'),
                                                                   mean_opening_balance=('openingbalanceamt', 'mean'),
                                                                   total_current_balance=('currentbalanceamt', 'sum'),
                                                                   mean_current_balance=('currentbalanceamt', 'mean'),
                                                                   total_monthly_installment=('monthlyinstalmentamt', 'sum'),
                                                                   total_amount_overdue=('amountoverdue', 'sum'),
                                                                   active_tradelines_count=('accountno', 'count'))

    # Calculating limit utilization on the open accounts
    df_tradeline_features_v2['limit_utilization'] = round(df_tradeline_features_v2.total_current_balance/
                                                           df_tradeline_features_v2.total_opening_balance, 2)

    df_tradeline_features_reqd = df_tradeline_features_v1.merge(df_tradeline_features_v2, on="application_id",
                                                                  how="left")
    
    return df_tradeline_features_reqd

#### Testing Helper Function

In [478]:
tradeline_features = extract_tradeline_data(formatted_base_credit_report_data)
tradeline_features

,application_id,highest_dpd_overall,highest_dpd_last_3_months,highest_dpd_last_6_months,cycle_count_dpd_greater_than_7,cycle_count_dpd_greater_than_15,tradeline_count,types_of_loans,total_unique_lenders,avg_loan_duration_days,bureau_account_age_years,total_opening_balance,mean_opening_balance,total_current_balance,mean_current_balance,total_monthly_installment,total_amount_overdue,active_tradelines_count,limit_utilization
0,9711360,36.0,3.0,3.0,4.0,3.0,7,5,2,1775.75,7.0,895179.98,179035.996000,145887.79,29177.558,60368.39,22563.7,5,0.16
1,9714953,40.0,0.0,5.0,7.0,1.0,17,5,4,414.40,8.0,148000.00,37000.000000,294770.06,73692.515,22223.13,0.0,4,1.99
2,9714978,109.0,0.0,109.0,1.0,1.0,3,2,2,187.50,1.0,1090936.00,363645.333333,110919.00,36973.000,7000.00,12000.0,3,0.10


### Final Function to generate all features 

The below function will take the json file location as input, perform feature engineering on it and return the features as output

In [480]:
def generate_features_from_credit_report(file_location):
    """This function takes the file location as input and returns a dataframe with all derived features"""
    
    # Read the json file from location given
    base_credit_report_data = pd.read_json(file_location)

    # Format the dataframe
    formatted_base_credit_report_data = pd.DataFrame()
    for i in range(len(base_credit_report_data)):
        # Creating an intermediate dataframe with all the keys in the data dictionary
        temp_df_1 = pd.DataFrame(base_credit_report_data.data.iloc[i])
        temp_df_1['application_id'] = base_credit_report_data.application_id.iloc[i]
    
        # Pivoting the intermediate dataframe to create a single row for every application_id
        temp_df_2 = temp_df_1.reset_index().pivot(columns=['index'],
                                                    index=['application_id'],
                                                    values=['consumerfullcredit']).reset_index()
        temp_df_2.columns = ['application_id'] + temp_df_2['consumerfullcredit'].keys().tolist()
    
        formatted_base_credit_report_data = pd.concat([formatted_base_credit_report_data, temp_df_2])
    
    # 1. Derive Personal and Demographic Information
    personal_demo_info = extract_personal_demographic_info(formatted_base_credit_report_data)

    # 2. Fetching Delinquency and Negative Credit Events
    credit_summary_delinquent_data = extract_credit_summary_delinquency_data(formatted_base_credit_report_data)

    # 3. Generating Credit Seeking Behavior Features
    enquiry_data = extract_enquiry_data(formatted_base_credit_report_data)

    # 4. Engineer Tradeline and Repayment Behavior Features
    tradeline_features = extract_tradeline_data(formatted_base_credit_report_data)
    
    feature_df = formatted_base_credit_report_data[['application_id']].copy()
    feature_df = feature_df.merge(personal_demo_info, on="application_id", how="left")
    feature_df = feature_df.merge(credit_summary_delinquent_data, on="application_id", how="left")
    feature_df = feature_df.merge(enquiry_data, on="application_id", how="left")
    feature_df = feature_df.merge(tradeline_features, on="application_id", how="left")

    return feature_df

#### Testing the final function

In [483]:
feature_df = generate_features_from_credit_report('../data/credit_report_sample.json')
feature_df

,application_id,gender,dependants,propertyownedtype,age,is_foreigner,count_of_occupations_listed,occupation_type,noofotheraccountsbad,noofotheraccountsgood,...,avg_loan_duration_days,bureau_account_age_years,total_opening_balance,mean_opening_balance,total_current_balance,mean_current_balance,total_monthly_installment,total_amount_overdue,active_tradelines_count,limit_utilization
0,9711360,Male,0,None,34.0,0,2,PUBLIC SERVANTS,0,3,...,1775.75,7.0,895179.98,179035.996000,145887.79,29177.558,60368.39,22563.7,5,0.16
1,9714953,Female,0,None,40.0,0,2,CIVIL SERVANT,0,3,...,414.40,8.0,148000.00,37000.000000,294770.06,73692.515,22223.13,0.0,4,1.99
2,9714978,Female,0,None,42.0,0,1,STUDENT,0,1,...,187.50,1.0,1090936.00,363645.333333,110919.00,36973.000,7000.00,12000.0,3,0.10


In [482]:
feature_df.columns

Index(['application_id', 'gender', 'dependants', 'propertyownedtype', 'age',
       'is_foreigner', 'count_of_occupations_listed', 'occupation_type',
       'noofotheraccountsbad', 'noofotheraccountsgood',
       'noofretailaccountsbad', 'noofretailaccountsgood',
       'nooftelecomaccountsbad', 'noofautoloanaccountsbad',
       'noofautoloanccountsgood', 'noofhomeloanaccountsbad',
       'nooftelecomaccountsgood', 'noofhomeloanaccountsgood',
       'noofjointloanaccountsbad', 'noofstudyloanaccountsbad',
       'noofcreditcardaccountsbad', 'noofjointloanaccountsgood',
       'noofstudyloanaccountsgood', 'noofcreditcardaccountsgood',
       'noofpersonalloanaccountsbad', 'noofpersonalloanaccountsgood', 'rating',
       'amountarrear', 'totalaccounts', 'totalaccountarrear',
       'totaloutstandingdebt', 'totaldishonouredamount',
       'totalmonthlyinstalment', 'is_currently_delinquent', 'subscribername',
       'monthsinarrears', 'total_enquiries', 'unique_subscriber_count',
       'co

## Summary of Engineered Features

I have engineered a total of 58 features, which can be grouped into the following categories based on the business logic they represent:

|*Category*|*# of Features*|*Example Features*|
|---|---|---|
|Demographics & Stability|7|age, dependants, is_foreigner|
|Delinquency & Negative Events|26|noofotheraccountsbad, noofcreditcardaccountsbad, totaloutstandingdebt, is_currently_delinquent|
|Credit Seeking Behavior|5|count_of_enquiries_last_3_months, days_since_last_enquiry, total_enquiries|
|Tradeline & Repayment Behavior|18|highest_dpd_last_3_months, total_active_accounts, bureau_account_age_years, limit_utilization|

#### Key Feature Spotlight & Business Rationale

While all engineered features contribute to the model, I want to highlight four "star players" that I hypothesize will have a particularly strong predictive power, based on my experience in the lending domain.
1. ___highest_dpd_last_6_months___ (A Measure of Recent Delinquency)
Rationale: This feature is arguably one of the most powerful leading indicators of default. A simple "yes/no" for delinquency is useful, but the severity of that delinquency (e.g., 90 days past due vs. 10 days past due) provides a much richer signal. Focusing on the last 6 months captures the customer's current financial health and discipline, making it more relevant than older, historical delinquencies.
2. ___limit_utilization_ratio___ (A Measure of Credit Dependency)
Rationale: This feature, often calculated as (total_current_balance / total_credit_limit), reflects how heavily a customer relies on credit. A consistently high utilization ratio suggests a customer may be living close to their financial limits, with little buffer to absorb unexpected financial shocks. This makes them inherently riskier than a customer who uses their available credit sparingly.
3. ___count_of_enquiries_last_3_months___ (A Measure of Credit Seeking Behavior)
Rationale: This is a classic behavioral feature that signals urgency. A sudden spike in credit applications within a short timeframe often indicates that an individual is facing an immediate financial need or has been rejected by other lenders. This "credit shopping" behavior is highly correlated with short-term default risk.
4. ___occupation_type___ (A Proxy for Income Stability)
Rationale: While demographic data is often secondary to behavioral data, occupation_type provides a valuable proxy for income stability and predictability. For instance, a salaried employee often has a more predictable cash flow than a self-employed individual, which can directly influence their ability to make consistent repayments. This feature helps the model differentiate between different risk archetypes.

#### Considerations for Modeling

With 58 features, it is likely that some will be highly correlated. For a final production model, a next step would be to run a correlation analysis (e.g., a heatmap) and consider dimensionality reduction techniques like VIF or using a model with built-in regularization (like Lasso) to select the most impactful features. 